In [3]:
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !pip install sentence-transformers
# !pip install ultralytics
# !pip install nltk

In [63]:
# !pip install groundingdino-py

In [64]:
# !pip install opencv-python pillow

In [4]:
# !pip install -U clip-interrogator


In [2]:
# from PIL import Image
# from clip_interrogator import Config, Interrogator

# config = Config(clip_model_name="ViT-B-32/openai")  # small and fast
# ci = Interrogator(config)

In [3]:
# image = Image.open(image_path).convert("RGB")
# description = ci.interrogate(image)

# print(description)

In [5]:
# from transformers import CLIPProcessor, CLIPModel
# from PIL import Image
# import numpy as np

In [92]:
clip_model_id = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(clip_model_id)
clip_processor = CLIPProcessor.from_pretrained(clip_model_id)


c:\InstalledApps\miniconda3\envs\my\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\1039682\.cache\huggingface\hub\models--openai--clip-vit-base-patch32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regul

In [93]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model.to(device)

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [ ]:
# def encode_image(image_path: str):
#     """Encode an image into a CLIP embedding vector."""
#     image = Image.open(image_path).convert("RGB")
#     inputs = clip_processor(images=image, return_tensors="pt").to(device)
#     with torch.no_grad():
#         image_features = clip_model.get_image_features(**inputs)
#     image_features /= image_features.norm(dim=-1, keepdim=True)
#     return image_features.cpu().numpy()[0]


# def encode_texts(texts: list[str]):
#     """Encode a list of text strings into CLIP embedding vectors."""
#     inputs = clip_processor(text=texts, return_tensors="pt", padding=True).to(device)
#     with torch.no_grad():
#         text_features = clip_model.get_text_features(**inputs)
#     text_features /= text_features.norm(dim=-1, keepdim=True)
#     return text_features.cpu().numpy()


In [6]:
# import torch

In [ ]:
# from transformers import AutoProcessor, AutoModel, AutoModelForZeroShotObjectDetection
# grounding_dino_processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-tiny")
# grounding_dino_model = AutoModelForZeroShotObjectDetection.from_pretrained("IDEA-Research/grounding-dino-tiny")

Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [89]:
# def detect_grounding_dino(image_path: str, question: str, box_threshold: float = 0.15, text_threshold: float = 0.15):
#     """
#     Detects objects in an image using Grounding DINO, guided by the question text.
#     Automatically downloads model weights from Hugging Face.
#     Returns: list[str] of detected object names.
#     """
#     image = Image.open(image_path).convert("RGB")

#     # Convert question to text prompt
#     # You can optionally extract key nouns instead of using full question
#     text_labels = question.lower().strip().split()
    
#     inputs = grounding_dino_processor(images=image, text=text_labels, return_tensors="pt")
#     with torch.no_grad():
#         outputs = grounding_dino_model(**inputs)

#     # Postprocess predictions
#     results = grounding_dino_processor.post_process_grounded_object_detection(
#         outputs,
#         inputs.input_ids,
#         threshold=box_threshold,
#         text_threshold=text_threshold,
#         target_sizes=[image.size[::-1]]
#     )

#     print(f"Results: {results}")
#     detected_objects = []
    
#     for pred in results:
#         for label in pred["labels"]:
#             detected_objects.append(label)

#     return list(set(detected_objects))


In [90]:
# detect_grounding_dino(image_path=image_path, question=question)

In [1]:
import requests
import spacy
from sentence_transformers import SentenceTransformer, util
import torch
import requests
from functools import lru_cache
import spacy
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
from ultralytics import YOLO
from collections import defaultdict
import os
import shutil
import os
import json
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image
from transformers import AutoProcessor, AutoModelForVision2Seq, pipeline
from tqdm import tqdm

c:\InstalledApps\miniconda3\envs\my\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from load_aokvqa import load_aokvqa, get_coco_path

In [3]:
# Download resources once
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

STOPWORDS = set(stopwords.words('english'))
LEMMATIZER = WordNetLemmatizer()
OBJECT_DETECTION_CONFIDENCE = 0.35  # threshold for detection pipeline


In [4]:
import random
import numpy as np
import torch

def set_seed(seed: int = 42):
    """
    Set random seed for reproducibility across Python, NumPy, and PyTorch.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # For deterministic behavior (slower but fully reproducible)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [5]:
nlp = spacy.load("en_core_web_sm")
embedder = SentenceTransformer("all-MiniLM-L6-v2")  # fast, small, good quality
# Load pretrained detector
detector = YOLO('yolov8s.pt')  # small but good baseline



In [ ]:
# def detect_objects_in_image(image: Image.Image, min_score=OBJECT_DETECTION_CONFIDENCE, max_objects=10):
#     """
#     Use HF object-detection pipeline (DETR) to get labels from the image.
#     Returns list of (label, score).
#     """
#     # pipeline will auto-download a model first run. Use default 'facebook/detr-resnet-50'
#     obj_pipeline = pipeline("object-detection", model="facebook/detr-resnet-50", device=0 if DEVICE=="cuda" else -1)
#     detections = obj_pipeline(image, threshold=min_score)
#     # results are list of dicts with 'label', 'score', 'box'
#     labels = []
#     for obj in detections[:max_objects]:
#         labels.append((obj["label"], float(obj["score"])))
#     # deduplicate while preserving order (higher score first)
#     seen = set()
#     out = []
#     for label, score in labels:
#         l = label.lower()
#         if l not in seen:
#             seen.add(l)
#             # out.append((l, score))
#             out.append(l)
#     return out


In [7]:

# ✅ ALL 34 ConceptNet Relations
RELATION_TEMPLATES = {
    "RelatedTo": "{a} is related to {b}.",
    "FormOf": "{a} is an inflected form of {b}.",
    "IsA": "{a} is a type of {b}.",
    "PartOf": "{a} is part of {b}.",
    "HasA": "{a} has a {b}.",
    "UsedFor": "{a} is used for {b}.",
    "CapableOf": "{a} is capable of {b}.",
    "AtLocation": "{a} is typically found at {b}.",
    "Causes": "{a} can cause {b}.",
    "HasSubevent": "{a} includes the event {b}.",
    "HasFirstSubevent": "{a} begins with {b}.",
    "HasLastSubevent": "{a} ends with {b}.",
    "HasPrerequisite": "{a} requires {b} to happen first.",
    "HasProperty": "{a} has the property of being {b}.",
    "MotivatedByGoal": "{a} is done in order to {b}.",
    "ObstructedBy": "{a} can be obstructed by {b}.",
    "Desires": "{a} desires {b}.",
    "CreatedBy": "{a} is created by {b}.",
    "Synonym": "{a} and {b} have similar meanings.",
    "Antonym": "{a} and {b} are opposites.",
    "DistinctFrom": "{a} is distinct from {b}.",
    "DerivedFrom": "{a} is derived from {b}.",
    "SymbolOf": "{a} symbolizes {b}.",
    "DefinedAs": "{a} is defined as {b}.",
    "MannerOf": "{a} is a way of doing {b}.",
    "LocatedNear": "{a} is located near {b}.",
    "HasContext": "{a} is used in the context of {b}.",
    "SimilarTo": "{a} is similar to {b}.",
    "EtymologicallyRelatedTo": "{a} and {b} share a common etymology.",
    "EtymologicallyDerivedFrom": "{a} is etymologically derived from {b}.",
    "CausesDesire": "{a} makes someone want {b}.",
    "MadeOf": "{a} is made of {b}.",
    "ReceivesAction": "{a} can be {b}.",
    "ExternalURL": "For more information about {a}, see {b}."
}

len(RELATION_TEMPLATES)

34

In [29]:
@lru_cache(maxsize=500000)
def query_conceptnet_triples(term, max_results=5, lang="en", return_dict=False):
    """
    Query ConceptNet for all relations of a given term.
    Returns natural-language statements or structured triples.

    Args:
        term (str): The concept to query.
        max_results (int): Max number of results to return.
        lang (str): Language code (default: 'en').
        return_dict (bool): If True, returns structured triples instead of text.

    Returns:
        list: List of sentences or dicts representing ConceptNet triples.
    """
    term = term.strip().lower().replace(" ", "_")
    url = f"http://api.conceptnet.io/c/{lang}/{term}"

    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        data = response.json()
    except Exception as e:
        print(f"[Warning] Error querying ConceptNet for '{term}': {e}")
        return []

    results = []
    for edge in data.get("edges", []):
        rel = edge.get("rel", {}).get("label", "")
        if rel not in RELATION_TEMPLATES:
            continue

        start = edge["start"].get("label", "")
        end = edge["end"].get("label", "")
        if not start or not end or start.lower() == end.lower():
            continue

        # Handle direction (ConceptNet sometimes reverses)
        if f"/c/{lang}/{term}" == edge["end"].get("@id", ""):
            start, end = end, start  # flip direction

        template = RELATION_TEMPLATES[rel]
        sentence = template.format(a=start, b=end)

        triple = {
            "subject": start,
            "relation": rel,
            "object": end,
            "sentence": sentence
        }

        # results.append(triple if return_dict else sentence)
        results.append(triple)

        # if len(results) >= max_results:
        #     break

    return results

In [9]:
# ✅ Example Usage
terms = ["cat", "sleep", "bridge"]
for t in terms:
    print(f"\n🔹 Facts about '{t}':")
    for fact in query_conceptnet_triples(t, max_results=7):
        print("  -", fact)



🔹 Facts about 'cat':
  - {'subject': 'a cat', 'relation': 'AtLocation', 'object': 'my lap', 'sentence': 'a cat is typically found at my lap.'}
  - {'subject': 'a cat', 'relation': 'AtLocation', 'object': 'a bed', 'sentence': 'a cat is typically found at a bed.'}
  - {'subject': 'a cat', 'relation': 'AtLocation', 'object': 'the windowsill', 'sentence': 'a cat is typically found at the windowsill.'}
  - {'subject': 'Cat', 'relation': 'CapableOf', 'object': 'hunt mice', 'sentence': 'Cat is capable of hunt mice.'}
  - {'subject': 'A cat', 'relation': 'HasA', 'object': 'four legs', 'sentence': 'A cat has a four legs.'}
  - {'subject': 'a cat', 'relation': 'CapableOf', 'object': 'drink water', 'sentence': 'a cat is capable of drink water.'}
  - {'subject': 'a cat', 'relation': 'CapableOf', 'object': 'catch a mouse', 'sentence': 'a cat is capable of catch a mouse.'}
  - {'subject': 'cat', 'relation': 'RelatedTo', 'object': 'feline', 'sentence': 'cat is related to feline.'}
  - {'subject': 'c

In [ ]:
def extract_keywords(question: str, max_terms: int = 3) -> list[str]:
    """
    Extracts key content words (mainly nouns) from the question.
    Cleans, lemmatizes, and removes stopwords/punctuation/numbers.
    Returns up to `max_terms` unique keywords.
    """
    if not isinstance(question, str):
        raise ValueError(f"Expected string input, got {type(question)}")

    doc = nlp(question)

    # Collect candidate tokens: prioritize noun chunks first
    candidates = []
    for chunk in doc.noun_chunks:
        root = chunk.root.lemma_.lower().strip()
        if root.isalpha() and root not in STOP_WORDS:
            candidates.append(root)

    # If too few candidates, add additional strong content words (nouns/adjectives)
    if len(candidates) < max_terms:
        for token in doc:
            if (
                token.pos_ in {"NOUN", "PROPN", "ADJ"} and
                token.is_alpha and
                token.lemma_.lower() not in STOP_WORDS
            ):
                candidates.append(token.lemma_.lower())

    # Deduplicate while preserving order
    cleaned = list(dict.fromkeys(candidates))

    return cleaned[:max_terms]


In [11]:
def rank_triples_by_relevance(question: str, triples: list[dict], top_k: int = 5) -> list[dict]:
    """
    Rank ConceptNet triples by cosine similarity between question and triple sentence.
    """
    if not triples:
        return []

    question_emb = embedder.encode(question, convert_to_tensor=True)
    triple_sents = [t["sentence"] for t in triples]
    triple_embs = embedder.encode(triple_sents, convert_to_tensor=True)

    scores = util.cos_sim(question_emb, triple_embs)[0]
    top_indices = torch.topk(scores, min(top_k, len(scores))).indices.tolist()

    ranked = [triples[i] for i in top_indices]
    for i, t in enumerate(ranked):
        t["similarity"] = float(scores[top_indices[i]])
    return ranked


In [12]:
def get_knowledge_context(question: str, top_k: int = 5) -> dict:
    """
    Retrieve and rank ConceptNet triples by relevance to the question.
    """
    print(f"Question: {question}")
    terms = extract_keywords(question)
    all_triples = []

    for term in terms:
        all_triples.extend(query_conceptnet_triples(term))

    # Deduplicate triples by (subject, relation, object)
    seen = set()
    unique_triples = []
    for t in all_triples:
        # print(f"Retrieved triple: {t}")
        key = (t["subject"].lower(), t["relation"], t["object"].lower())
        if key not in seen:
            seen.add(key)
            unique_triples.append(t)

    # Rank by semantic similarity
    ranked_triples = rank_triples_by_relevance(question, unique_triples, top_k=top_k)

    # Build context string
    sentences = " ".join([t["sentence"] for t in ranked_triples])

    return {
        "triples": ranked_triples,
        "sentences": sentences
    }


In [ ]:
# def detect_objects(image_path, conf_threshold=0.3, max_objects=10):
#     """
#     Detect objects in the image and return class labels.
#     """
#     results = detector(image_path)
#     labels = []
#     for r in results:
#         for c in r.boxes.cls:
#             labels.append(results[0].names[int(c)])
#     return list(dict.fromkeys(labels))[:max_objects]

# def detect_objects(image_path):
#     return detect_objects_in_image(Image.open(image_path))

def detect_objects(image_path, conf_threshold=0.3, max_objects=10):
    """
    Detect objects in the image and return class labels.
    """
    # Run detection with confidence threshold
    results = detector(image_path, conf=conf_threshold)

    labels = []
    for r in results:
        # Filter boxes by confidence
        for box in r.boxes:
            if box.conf >= conf_threshold:
                class_id = int(box.cls)
                labels.append(r.names[class_id])

    # Remove duplicates and limit to max_objects
    unique_labels = list(dict.fromkeys(labels))[:max_objects]
    return unique_labels

In [15]:
def expand_conceptnet_terms(term, max_neighbors=3):
    """
    Get semantically related terms for a given term from ConceptNet.
    Used as a fallback when no relevant terms found.
    """
    neighbors = []
    url = f"http://api.conceptnet.io/query?node=/c/en/{term}&rel=/r/RelatedTo&limit={max_neighbors}"
    try:
        import requests
        response = requests.get(url).json()
        for edge in response.get("edges", []):
            other = edge["end"]["label"] if edge["start"]["label"].lower() == term.lower() else edge["start"]["label"]
            neighbors.append(other.lower())
    except Exception:
        pass
    return list(set(neighbors))

def get_visual_question_terms(image_path, question, intersection_first=True, expand_if_empty=True):
    """
    Combines visual and question-derived terms, robustly handling all empty cases.
    Returns relevant_terms (for retrieval), plus raw question_terms and visual_terms.
    """
    print(f"Question: {question}")
    question_terms = extract_keywords(question)
    visual_terms = detect_objects(image_path)

    # Step 1: intersection (precise)
    if intersection_first:
        relevant_terms = list(set(question_terms) & set(visual_terms))
    else:
        relevant_terms = list(set(question_terms) | set(visual_terms))

    # Step 2: if intersection empty, fall back to union
    if not relevant_terms:
        relevant_terms = list(set(question_terms) | set(visual_terms))

    # Step 3: if still empty, use ConceptNet expansion of question terms
    if expand_if_empty and not relevant_terms and question_terms:
        expanded = []
        for qt in question_terms:
            expanded.extend(expand_conceptnet_terms(qt))
        relevant_terms = list(set(question_terms + expanded))

    # Step 4: final safety net
    if not relevant_terms:
        relevant_terms = question_terms if question_terms else [question]

    return relevant_terms, question_terms, visual_terms


In [16]:
def sanitize_terms(terms, max_words=3):
    """
    Clean and filter terms before querying ConceptNet.
    Includes stopword removal and lemmatization.
    """
    clean_terms = []
    
    for t in terms:
        if not isinstance(t, str): 
            continue
        
        # Lowercase + remove punctuation
        t = t.strip().lower()
        t = re.sub(r"[^\w\s]", "", t)
        
        # Tokenize
        tokens = t.split()
        
        # Remove stopwords and lemmatize
        tokens = [LEMMATIZER.lemmatize(tok) for tok in tokens if tok not in STOPWORDS]
        
        # Skip if empty or numeric
        if not tokens or all(tok.isnumeric() for tok in tokens):
            continue
        
        # Rejoin meaningful multi-word concepts
        processed = " ".join(tokens)
        
        # Skip phrases that are too long
        if len(tokens) > max_words:
            continue
        
        clean_terms.append(processed)
    
    # Deduplicate while preserving order
    return list(dict.fromkeys(clean_terms))


In [17]:
def get_grounded_knowledge_context(image_path, question, top_k=5):
    relevant_terms, q_terms, v_terms = get_visual_question_terms(image_path, question)
    
    # 🔧 Sanitize terms before retrieval
    # Use improved sanitization
    sanitized_terms = sanitize_terms(relevant_terms)

    if not sanitized_terms:
        sanitized_terms = sanitize_terms(q_terms)
    if not sanitized_terms:
        sanitized_terms = [question.split()[0]]  # last-resort fallback
    
    print(f"Question terms: {q_terms}")
    print(f"Visual terms: {v_terms}")
    print(f"Retrieving ConceptNet for: {sanitized_terms}")
    
    all_triples = []
    for term in sanitized_terms:
        triples = query_conceptnet_triples(term, max_results=10)
        if triples:
            all_triples.extend(triples)

    # Deduplicate
    seen = set()
    unique_triples = []
    for t in all_triples:
        key = (t['subject'].lower(), t['relation'], t['object'].lower())
        if key not in seen:
            seen.add(key)
            unique_triples.append(t)

    # If still no triples, gracefully degrade to linguistic similarity
    if not unique_triples:
        print("⚠️ No ConceptNet triples found, using question terms as pseudo-context.")
        sentences = f"The question is about {', '.join(sanitized_terms)}."
        return {
            "triples": [],
            "sentences": sentences,
            "visual_terms": v_terms,
            "question_terms": q_terms
        }

    # Rank triples normally
    ranked = rank_triples_by_relevance(question, unique_triples, top_k=top_k)
    sentences = " ".join([t["sentence"] for t in ranked])

    return {
        "triples": ranked,
        "sentences": sentences,
        "visual_terms": v_terms,
        "question_terms": q_terms
    }


In [ ]:
# def build_grounded_prompt(image_path, question, choices):
#     context = get_grounded_knowledge_context(image_path, question, top_k=5)

#     prompt = f"""
#     You are a visual QA model. Use the image (via detected objects) as primary evidence; use the provided knowledge **only** as a supporting signal. Do not invent facts.

# Relevant Knowledge:
# {context['sentences']}

# Question:
# {question}

# Answer choices:
# {', '.join(choices)}

# Task:
# Choose the single best answer from the choices. Basis your choice on visible evidence first; use knowledge only when it increases confidence. If no choice is supported by the image or knowledge, pick the least-incorrect option.

# **Respond only with the final answer choice.**
#     """
#     return prompt.strip()


In [18]:
def build_grounded_prompt(image_path, question, choices):
    context = get_grounded_knowledge_context(image_path, question, top_k=5)

    prompt = f"""
    You are a visual QA model. Use detected objects as the primary evidence; use the provided knowledge only as a secondary supporting signal. Do not invent facts or add unseen details.

    Relevant Knowledge:
    {context['sentences']}

    Question:
    {question}

    Answer choices:
    {', '.join(choices)}

    Task:
    Pick the single best answer
    """
    return prompt.strip()


In [19]:
aokvqa_dir = os.getenv('AOKVQA_DIR', r"C:\workspace\misc\5980\aokvqa")
coco_dir = os.getenv('COCO_DIR', r"C:\workspace\misc\5980\coco")
coco_filtered_dir = os.getenv('COCO_FILTERED_DIR', r"C:\workspace\misc\5980\coco_filtered")

In [20]:
val_aokvqa_dataset = load_aokvqa(aokvqa_dir, 'val')  
train_aokvqa_dataset = load_aokvqa(aokvqa_dir, 'train')  
test_aokvqa_dataset = load_aokvqa(aokvqa_dir, 'test')

In [21]:
print(f"Train dataset size: {len(train_aokvqa_dataset)}")
print(f"Validation dataset size: {len(val_aokvqa_dataset)}")
print(f"Test dataset size: {len(test_aokvqa_dataset)}")

Train dataset size: 17056
Validation dataset size: 1145
Test dataset size: 6702


In [22]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

Using device: cpu


In [28]:
image_id = 461751
question = "What is in the motorcyclist's mouth?"
choices = [
    "toothpick",
    "food",
    "popsicle stick",
    "cigarette"
]
image_path = get_coco_path("val", image_id, coco_dir)
image_path

'C:\\workspace\\misc\\5980\\coco\\val2017\\000000461751.jpg'

In [59]:
grounded_prompt = build_grounded_prompt(
    image_path=image_path,
    question=question,
    choices=choices
)

Question: What is in the motorcyclist's mouth?



image 1/1 C:\workspace\misc\5980\coco\val2017\000000461751.jpg: 576x640 4 persons, 1 car, 1 motorcycle, 252.7ms
Speed: 10.0ms preprocess, 252.7ms inference, 3.5ms postprocess per image at shape (1, 3, 576, 640)
Question terms: ['mouth', 'motorcyclist']
Visual terms: ['person', 'car', 'motorcycle']
Retrieving ConceptNet for: ['mouth', 'car', 'motorcyclist', 'motorcycle', 'person']


In [60]:
print(grounded_prompt)

You are a visual QA model. Use detected objects as the primary evidence; use the provided knowledge only as a secondary supporting signal. Do not invent facts or add unseen details.

    Relevant Knowledge:
    motorcyclist is derived from motorcycle. motorcyclist is related to motorradfahrerin. motorcyclist and biker have similar meanings. motorcyclist is related to motociclista. mouth is related to part.

    Question:
    What is in the motorcyclist's mouth?

    Answer choices:
    toothpick, food, popsicle stick, cigarette

    Task:
    Pick the single best answer


In [24]:
# grounded_prompts = {}

# for data_point in tqdm(val_aokvqa_dataset):
#     image_path = get_coco_path("val", data_point['image_id'], coco_dir)
#     grd_prompt = build_grounded_prompt(
#         image_path=image_path,
#         question=data_point['question'],
#         choices=data_point['choices']
#     )
#     grounded_prompts[data_point['image_id']] = grd_prompt



In [31]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def build_prompt_for_data_point(data_point):
    image_path = get_coco_path("val", data_point['image_id'], coco_dir)
    grd_prompt = build_grounded_prompt(
        image_path=image_path,
        question=data_point['question'],
        choices=data_point['choices']
    )
    return data_point['image_id'], grd_prompt

def generate_grounded_prompts_parallel(dataset, max_workers=8):
    grounded_prompts = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(build_prompt_for_data_point, dp) for dp in dataset]
        for future in tqdm(as_completed(futures), total=len(futures)):
            image_id, grd_prompt = future.result()
            grounded_prompts[image_id] = grd_prompt
    return grounded_prompts

# Example usage:
max_workers = 8  # Set this to your desired level of parallelism
grounded_prompts = generate_grounded_prompts_parallel(val_aokvqa_dataset, max_workers=max_workers)

Question: What is in the motorcyclist's mouth?
Question: Which number birthday is probably being celebrated?
Question: What best describes the pool of water?
Question: What is the white substance on top of the cupcakes?
Question: What type of device is sitting next to the laptop?
Question: The thing on the animal to the left's head is similar to what is on the head of what else?
Question: What is the appliance the woman is holding used for?
Question: What is the descriptive word for this surface?


  0%|          | 0/1145 [00:00<?, ?it/s]









image 1/1 C:\workspace\misc\5980\coco\val2017\000000461751.jpg: 576x640 4 persons, 1 car, 1 motorcycle, 275.6ms
Speed: 7.8ms preprocess, 275.6ms inference, 1.8ms postprocess per image at shape (1, 3, 576, 640)
Question terms: ['mouth', 'motorcyclist']
Visual terms: ['person', 'car', 'motorcycle']
Retrieving ConceptNet for: ['person', 'mouth', 'car', 'motorcyclist', 'motorcycle']
image 1/1 C:\workspace\misc\5980\coco\val2017\000000198960.jpg: 448x640 4 sheeps, 158.2ms
Speed: 2.4ms preprocess, 158.2ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)
Question terms: ['thing', 'animal', 'head']
Visual terms: ['sheep']
Retrieving ConceptNet for: ['animal', 'sheep', 'head', 'thing']
image 1/1 C:\workspace\misc\5980\coco\val2017\000000377368.jpg: 512x640 1 person, 2 cups, 2 cakes, 2 dining tables, 187.3ms
Speed: 4.1ms preprocess, 187.3ms inference, 1.8ms postprocess per image at shape (1, 3, 512, 640)
Question terms: ['birthday', 'number']
Visual terms: ['cup', 'person

  0%|          | 1/1145 [00:15<4:47:19, 15.07s/it]

Question: What is the person on the left doing with their body?

image 1/1 C:\workspace\misc\5980\coco\val2017\000000341196.jpg: 480x640 3 persons, 1 surfboard, 157.6ms
Speed: 2.9ms preprocess, 157.6ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)


...

 99%|█████████▉| 1137/1145 [11:16<00:02,  3.52it/s]

Question: What does the yellow sign advise you to watch for?
image 1/1 C:\workspace\misc\5980\coco\val2017\000000313588.jpg: 640x480 1 person, 1 car, 1 truck, 1 umbrella, 1 handbag, 219.3ms
Speed: 1.5ms preprocess, 219.3ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 480)
Question terms: ['pattern', 'coat', 'woman']
Visual terms: ['person', 'car', 'umbrella', 'truck', 'handbag']
Retrieving ConceptNet for: ['person', 'woman', 'pattern', 'umbrella', 'truck', 'handbag', 'coat', 'car']



 99%|█████████▉| 1138/1145 [11:16<00:01,  3.95it/s]

image 1/1 C:\workspace\misc\5980\coco\val2017\000000369751.jpg: 640x448 1 person, 3 cars, 1 stop sign, 4 parking meters, 146.0ms
Speed: 1.8ms preprocess, 146.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 448)
Question terms: ['sign', 'yellow']
Visual terms: ['parking meter', 'stop sign', 'car', 'person']
Retrieving ConceptNet for: ['person', 'sign', 'stop sign', 'yellow', 'parking meter', 'car']


100%|██████████| 1145/1145 [11:19<00:00,  1.68it/s]


In [32]:
len(grounded_prompts)

1122

In [ ]:
# import pickle

# with open('grounded_prompts_dict.pkl', 'wb') as fp:
#     pickle.dump(grounded_prompts, fp)

In [46]:
# import pickle
# with open('data/val_predictions/val_predictions_mcq_grounded_prompts_SmolVLM-Instruct.pkl', 'rb') as fp:
#     a = pickle.load(fp)

In [61]:
model_name = "HuggingFaceTB/SmolVLM-256M-Instruct"
# model_name = "HuggingFaceTB/SmolVLM-500M-Instruct"
# model_name = "HuggingFaceTB/SmolVLM-Instruct"

# model_name = "HuggingFaceTB/SmolVLM-256M-Base"

Single Prediction

In [39]:
# Initialize processor and model
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    # _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
).to(DEVICE)

Fetching 2 files: 100%|██████████| 2/2 [00:00<?, ?it/s]


In [62]:
def get_prediction(data_point, coco_dir, split):
    question = data_point['question']

    image_path = get_coco_path(split, data_point['image_id'], coco_dir)
    # image = load_image(image_path)
    image = Image.open(image_path).convert("RGB")

    grounded_question = build_grounded_prompt(image_path, question, data_point['choices'])
    # Create input messages
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": grounded_question}
            ]
        },
    ]

    # Prepare inputs
    prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=prompt, images=[image], return_tensors="pt")
    inputs = inputs.to(DEVICE)

    # Generate outputs
    generated_ids = model.generate(**inputs, max_new_tokens=500)
    generated_texts = processor.batch_decode(
        generated_ids,
        skip_special_tokens=True,
    )
    return data_point, generated_texts[0]


In [63]:
predictions = []
for idx, data_point in enumerate(val_aokvqa_dataset):
    data_point, pred = get_prediction(data_point, coco_filtered_dir, 'val')
    print(idx, end=",")
    predictions.append((data_point, pred))
    break

Question: What is in the motorcyclist's mouth?



image 1/1 C:\workspace\misc\5980\coco_filtered\val2017\000000461751.jpg: 576x640 4 persons, 1 car, 1 motorcycle, 243.2ms
Speed: 3.9ms preprocess, 243.2ms inference, 1.8ms postprocess per image at shape (1, 3, 576, 640)
Question terms: ['mouth', 'motorcyclist']
Visual terms: ['person', 'car', 'motorcycle']
Retrieving ConceptNet for: ['mouth', 'car', 'motorcyclist', 'motorcycle', 'person']
0,

In [64]:
data_point, pred

({'split': 'val',
  'image_id': 461751,
  'question_id': '22jbM6gDxdaMaunuzgrsBB',
  'question': "What is in the motorcyclist's mouth?",
  'choices': ['toothpick', 'food', 'popsicle stick', 'cigarette'],
  'correct_choice_idx': 3,
  'direct_answers': ['cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette'],
  'difficult_direct_answer': False,
  'rationales': ["He's smoking while riding.",
   'The motorcyclist has a lit cigarette in his mouth while he rides on the street.',
   'The man is smoking.']},
 "User:\n\n\n\n\nYou are a visual QA model. Use detected objects as the primary evidence; use the provided knowledge only as a secondary supporting signal. Do not invent facts or add unseen details.\n\n    Relevant Knowledge:\n    motorcyclist is derived from motorcycle. motorcyclist is related to motorradfahrerin. motorcyclist and biker have similar meanings. motorcyclist is related to mot